## OpenSearch Hybrid 검색을 통한 RAG

#### 설정

In [8]:
import sys, os
module_path = ".."
sys.path.append(os.path.abspath(module_path))

### 1. Bedrock Client 생성

In [9]:
from utils import bedrock

boto3_bedrock = bedrock.get_bedrock_client(
  assumed_role=None,
  endpoint_url=None,
  region='us-east-1'
)

Create new client
  Using region: us-east-1
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)


### 2. Titan Embedding 및 LLM Claude-v2 모델 로딩

#### LLM 로딩

In [10]:
from langchain.llms import Bedrock
from langchain.callbacks.stdout import StdOutCallbackHandler

# Create Anthropic Model
llm_text = Bedrock(
  model_id = "anthropic.claude-v2",
  client = boto3_bedrock,
  model_kwargs={
    "max_tokens_to_sample": 512
  },
  streaming=True,
  callbacks=[StdOutCallbackHandler()]
)

#### Embedding Model 선택

In [11]:
from langchain.embeddings import BedrockEmbeddings

llm_emb = BedrockEmbeddings(
  model_id="amazon.titan-embed-text-v1",
  client=boto3_bedrock,
  region_name="us-east-1",
)

### 전 단계 02_1에서 생성한 reindex 한 것을 지워서 reindex 과정 반복

In [12]:
# opensearch info
host = "localhost"
port = 9200
opensearch_endpoint = f"https://{host}:{port}"
http_auth = ("admin", "admin")
index_name = "genai-demo-index-v1"

ca_certs_path = 'root-ca.pem'
# Optional client certificates if you don't want to use HTTP basic authentication.
client_cert_path = 'admin.pem'
client_key_path = 'admin-key.pem'

#### OpenSearch ReIndexig
- 기존 "index genai-demo-index-v1"을 "genai-demo-index-v1-with-tokenizer"로 
- nori tokenizer 추가

OpenSearch Client

In [13]:
from opensearchpy import OpenSearch

# Create the client with SSL/TLS enabled, but hostname verification disabled.
os_client = OpenSearch(
    hosts = [{'host': host, 'port': port}],
    http_compress = True, # enables gzip compression for request bodies
    http_auth = http_auth,
    client_cert = client_cert_path,
    client_key = client_key_path,
    use_ssl = True,
    verify_certs = True,
    ssl_assert_hostname = False,
    ssl_show_warn = False,
    ca_certs = ca_certs_path
)

In [14]:
from pprint import pprint

index_info = os_client.indices.get(index=index_name)
pprint(index_info)

{'genai-demo-index-v1': {'aliases': {},
                         'mappings': {'properties': {'metadata': {'properties': {'row': {'type': 'long'},
                                                                                 'source': {'fields': {'keyword': {'ignore_above': 256,
                                                                                                                   'type': 'keyword'}},
                                                                                            'type': 'text'},
                                                                                 'timestamp': {'type': 'float'},
                                                                                 'type': {'fields': {'keyword': {'ignore_above': 256,
                                                                                                                 'type': 'keyword'}},
                                                                                          't

In [ ]:
new_index_name = f"{index_name}-with-tokenizer"
new_index_name

'genai-demo-index-v1-with-tokenizer'

In [ ]:
# use nori tokenizer 사용
# analyzer_config = {
#   "tokenizer": "nori",
#   "tokenizer_type": "nori_tokenizer",
#   "char_filter": ["html_strip"],
#   "filter": ["nori_number", "nori_readingform", "lowercase"],
#   "decompound_mode": "mixed",
#   "discard_punctuation": "true"
# }

#### index 수정: 형태소 분석기 사용 enablement

In [ ]:
# nori 형태소 분석기를 custom analyzer 등록
index_info[index_name]["settings"]["analysis"] = {
  "tokenizer": {
    "nori": {
      "type": "nori_tokenizer",
      "decompound_mode": "mixed",
      "discard_punctuation": "true"
    }
  },
  "analyzer": {
    "my_analyzer": {
      "type": "custom",
      "char_filter": ["html_strip"],
      "tokenizer": "nori",
      "filter": ["nori_number", "nori_readingform", "lowercase"]  # token filter
    }
  }
}

# analyzer가 적용될 칼럼에 맞추어 수정
index_info[index_name]["mappings"]["properties"]["text"]["analyzer"] = "my_analyzer"
index_info[index_name]['mappings']['properties']['text']['search_analyzer'] = "my_analyzer"

# index 설정 변경 없음
index_info[index_name]["settings"]["index"] = {
  "number_of_shards": "5",
  "knn.algo_param": {"ef_search": "512"},
  "knn": True,
  "number_of_replicas": "2"
}

# del index alias
# del index_info[index_name]["aliases"]
new_index_info = index_info[index_name]


In [ ]:
pprint(new_index_info)

{'aliases': {},
 'mappings': {'properties': {'metadata': {'properties': {'row': {'type': 'long'},
                                                         'source': {'fields': {'keyword': {'ignore_above': 256,
                                                                                           'type': 'keyword'}},
                                                                    'type': 'text'},
                                                         'timestamp': {'type': 'float'},
                                                         'type': {'fields': {'keyword': {'ignore_above': 256,
                                                                                         'type': 'keyword'}},
                                                                  'type': 'text'}}},
                             'text': {'analyzer': 'my_analyzer',
                                      'fields': {'keyword': {'ignore_above': 256,
                                                    

In [ ]:
from utils.opensearch import opensearch_utils

index_exists = opensearch_utils.check_if_index_exists(os_client, new_index_name)
if index_exists:
    opensearch_utils.delete_index(os_client, new_index_name)
else:
    print("Index does not exist")

index_name=genai-demo-index-v1-with-tokenizer, exists=False
Index does not exist


In [ ]:
# create index
reponse = os_client.indices.create(new_index_name, body=new_index_info)
print(reponse)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'genai-demo-index-v1-with-tokenizer'}


In [ ]:
# reindex
_reindex = {
  "source": {"index": index_name},
  "dest": {"index": new_index_name}
}
print(_reindex)
response = os_client.reindex(_reindex)
print(response)

{'source': {'index': 'genai-demo-index-v1'}, 'dest': {'index': 'genai-demo-index-v1-with-tokenizer'}}
{'took': 470, 'timed_out': False, 'total': 92, 'updated': 0, 'created': 92, 'deleted': 0, 'batches': 1, 'version_conflicts': 0, 'noops': 0, 'retries': {'bulk': 0, 'search': 0}, 'throttled_millis': 0, 'requests_per_second': -1.0, 'throttled_until_millis': 0, 'failures': []}


## 3. LangChain OpenSearch VectorStore 생성

#### 새로 생성한 index name으로 변경

In [15]:
index_name = "genai-demo-index-v1-with-tokenizer"

In [16]:
from langchain.vectorstores import OpenSearchVectorSearch

vector_db = OpenSearchVectorSearch(
  opensearch_url=opensearch_endpoint,
  embedding_function=llm_emb,
  index_name =index_name,
  http_auth=http_auth,
  is_aoss=False,
  engine="faiss",
  space_type="l2",
  use_ssl=True,
  verify_certs=True,
  ssl_assert_hostname=False,
  ssl_show_warn=False,
  ca_certs=ca_certs_path
)


In [17]:
from pprint import pprint

index_info = os_client.indices.get(index_name)
pprint(index_info)

{'genai-demo-index-v1-with-tokenizer': {'aliases': {},
                                        'mappings': {'properties': {'metadata': {'properties': {'row': {'type': 'long'},
                                                                                                'source': {'fields': {'keyword': {'ignore_above': 256,
                                                                                                                                  'type': 'keyword'}},
                                                                                                           'type': 'text'},
                                                                                                'timestamp': {'type': 'float'},
                                                                                                'type': {'fields': {'keyword': {'ignore_above': 256,
                                                                                                                         

## 4. OpenSearch "유사서치" 검색
use similarity_search_with_score function

In [18]:
import copy
from langchain.schema import Document
from langchain import PromptTemplate
from operator import itemgetter
from langchain.chains.question_answering import load_qa_chain

### Question and Answering Chain 정의

### 1. OpenSearch Vector 검색 (Semantic Search)

#### prompt template 생성

In [19]:
from utils.rag import run_RetrievalQA, show_context_used
from langchain.prompts import PromptTemplate

In [20]:
prompt_template = """

Human: Here is the context, inside <context></context> XML tags.

<context>
{context}
</context>

Only using the context as above, answer the following question with the rule as below:
    - Don't insert XML tag such as <context> and </context> when answering.
    - Write as much as you can
    - Be courteous and polite
    - Only answer the question if you can find the answer in the context with certainty.

Question:
{question}

If the answer is not in the context, just say "주어진 내용에서 관련 답변을 찾을 수 없습니다."


Assistant:"""

PROMPT = PromptTemplate(
  template=prompt_template, input_variables=["context", "question"]
)
pprint(PROMPT)

PromptTemplate(input_variables=['context', 'question'], template='\n\nHuman: Here is the context, inside <context></context> XML tags.\n\n<context>\n{context}\n</context>\n\nOnly using the context as above, answer the following question with the rule as below:\n    - Don\'t insert XML tag such as <context> and </context> when answering.\n    - Write as much as you can\n    - Be courteous and polite\n    - Only answer the question if you can find the answer in the context with certainty.\n\nQuestion:\n{question}\n\nIf the answer is not in the context, just say "주어진 내용에서 관련 답변을 찾을 수 없습니다."\n\n\nAssistant:')


In [21]:
chain = load_qa_chain(
  llm=llm_text,
  chain_type="stuff",
  prompt=PROMPT,
  verbose=True
)

#### 필터 및 쿼리 생성

In [22]:
from utils.opensearch import opensearch_utils

filter01="홈페이지"
filter02="신한은행"

query="홈페이지 이용자아이디 여러 개 사용할 수 있나요?"

boolean_filter = opensearch_utils.get_filter(
  filter=[
    {"term": {"metadata.type": filter01}},
    {"term": {"metadata.source": filter02}},
  ]
)

pprint(boolean_filter)

{'bool': {'filter': [{'term': {'metadata.type': '홈페이지'}},
                     {'term': {'metadata.source': '신한은행'}}]}}


### Retriever for semantic search 정의

In [23]:
opensearch_semantic_retriever = vector_db.as_retriever(
  search_type="similarity",
  search_kwargs = {
    "k": 5,
    "boolean_filter": boolean_filter
  }
)

In [24]:
search_semantic_result = opensearch_semantic_retriever.get_relevant_documents(query)

answer = chain.run(
  input_documents=search_semantic_result,
  question=query
)

print("##############################")
print("query: \n", query)
print("answer: \n", answer)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:


Human: Here is the context, inside <context></context> XML tags.

<context>
ask: 홈페이지 이용자아이디 여러 개 사용할 수 있나요?
Information: 홈페이지 이용자 아이디는 개인의 경우 1인 1개만 이용 가능하고 기업의 경우에는 1개의 사업자번호당 사용자별로 이용자아이디를 복수로 사용할 수 있습니다. 
※ 개인사업자의 경우 개인과 기업 각각 이용자아이디를 발급하여 복수로 이용 가능합니다. 
기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다.

ask: 홈페이지 회원아이디를 변경하고 싶습니다.
Information: 회원탈퇴를 하시고, 사용하시고 싶은 아이디로 신규 회원가입신청을 하시면 됩니다. 다만, 사용하시고 싶은 아이디가 이미 다른 고객이 사용하고 있는 경우에는 사용하실 수 없습니다. 기타 문의는 콜센터 1599-8000번으로 문의 바랍니다.

ask: 만14세 미만의 고객은 홈페이지 회원가입이 가능하나요?
Information: 만 14세 미만 고객은 회원가입시 '정보통신망 이용촉진 및 정보 등에 관한 법률' 및 '개인정보보호지침'에 따라 법정대리인의 정보활용 동의가 필요합니다. 당행 영업점을 통한 보호자분의 홈페이지 회원 법정대리인 등록이 완료된 고객님에 한하여 회원등록이 가능하므로 홈페이지 회원 법정대리인 등록이 안된 고객님의 보호자분께서는 먼저 가까운 영업점을 방문하여 주시기 바랍니다. [구비서류] - 법정대리인 실명확인증표 - 미성년자 기준으로 발급된 특정 또는 상세 기본증명서 - 미성년자 기준으로 발급된 가족관계증명서 * 은행 방문일로부터 3개월 이내 발급한 서류로 준비해주셔야 하며, 성함과 주민등록번호 13자리가 모두 기재되어 있어야합니다. 기타 문의는

In [25]:
from utils.rag import show_context_used

In [26]:
show_context_used(search_semantic_result)

-----------------------------------------------
1. Chunk: 232 Characters
-----------------------------------------------
ask: 홈페이지 이용자아이디 여러 개 사용할 수 있나요?
Information: 홈페이지 이용자 아이디는 개인의 경우 1인 1개만 이용 가능하고 기업의 경우에는 1개의 사업자번호당 사용자별로 이용자아이디를 복수로 사용할 수 있습니다.
※ 개인사업자의 경우 개인과 기업 각각 이용자아이디를 발급하여 복수로 이용 가능합니다.
기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다.
metadata:
 {'source': '신한은행', 'row': 21, 'type': '홈페이지', 'timestamp': 1701838934.879154}
-----------------------------------------------
2. Chunk: 172 Characters
-----------------------------------------------
ask: 홈페이지 회원아이디를 변경하고 싶습니다.
Information: 회원탈퇴를 하시고, 사용하시고 싶은 아이디로 신규 회원가입신청을 하시면 됩니다. 다만, 사용하시고 싶은 아이디가 이미 다른 고객이 사용하고 있는 경우에는
사용하실 수 없습니다. 기타 문의는 콜센터 1599-8000번으로 문의 바랍니다.
metadata:
 {'source': '신한은행', 'row': 45, 'type': '홈페이지', 'timestamp': 1701838934.8792145}
-----------------------------------------------
3. Chunk: 436 Characters
-----------------------------------------------
ask: 만14세 미만의 고객은 홈페이지 회원가입이 가능하나요?
Information: 만 14세 미

### 2. OpenSearch Keyworkd 검색

In [27]:
from utils.rag import OpenSearchLexicalSearchRetriever

#### Retriver for lexical search 정의 (Keyword search)

In [28]:
opensearch_lexical_retriever = OpenSearchLexicalSearchRetriever(
  os_client=os_client,
  index_name=index_name
)

In [29]:
filter01 = "홈페이지"
#filter01 = "인증서"
filter02 = "신한은행"
# filter02 = "아마존은행"

query = "홈페이지 이용자아이디 여러 개 사용할 수 있나요?"
#query = "타기관OTP 등록 방법 알려주세요"

##### [TIP] lexical search의 parameter 변경이 필요한 경우, "update_search_params"를 활용
해당 함수는 search 함수(get_relevant_documents) 수행 시 Reset된다


In [30]:
opensearch_lexical_retriever.update_search_params(
  k=5,
  minimum_should_match=0,
  filter=[
    {"term": {"metadata.type": filter01}},
    {"term": {"metadata.source": filter02}},
  ],
)

search_keyword_result = opensearch_lexical_retriever.get_relevant_documents(query)

answer = chain.run(
  input_documents=search_keyword_result,
  question=query
)

print("##############################")
print("query: \n", query)
print("answer: \n", answer)

lexical search query: 
{'query': {'bool': {'filter': [{'term': {'metadata.type': '홈페이지'}},
                               {'term': {'metadata.source': '신한은행'}}],
                    'must': [{'match': {'text': {'minimum_should_match': '0%',
                                                 'operator': 'or',
                                                 'query': '홈페이지 이용자아이디 여러 개 '
                                                          '사용할 수 있나요?'}}}]}},
 'size': 5}


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:


Human: Here is the context, inside <context></context> XML tags.

<context>
ask: 홈페이지 이용자아이디 여러 개 사용할 수 있나요?
Information: 홈페이지 이용자 아이디는 개인의 경우 1인 1개만 이용 가능하고 기업의 경우에는 1개의 사업자번호당 사용자별로 이용자아이디를 복수로 사용할 수 있습니다. 
※ 개인사업자의 경우 개인과 기업 각각 이용자아이디를 발급하여 복수로 이용 가능합니다. 
기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다.

ask: 영업점에서 인터넷뱅킹 가입하고 왔는데 홈페이지회원은 별도로 가입해야 하나요?
Information: 인터넷뱅킹 가입과 홈페이지 회원은 개별로 운영되고 있습니다. 
인터넷뱅킹만 가입을 하

#### 키워드 검색 결과 (search_keyword_result)
bm25 score는 max_value로 normalizeation 되어 있음 (score range 0~1)

In [31]:
show_context_used(search_keyword_result)

-----------------------------------------------
1. Chunk: 232 Characters
-----------------------------------------------
ask: 홈페이지 이용자아이디 여러 개 사용할 수 있나요?
Information: 홈페이지 이용자 아이디는 개인의 경우 1인 1개만 이용 가능하고 기업의 경우에는 1개의 사업자번호당 사용자별로 이용자아이디를 복수로 사용할 수 있습니다.
※ 개인사업자의 경우 개인과 기업 각각 이용자아이디를 발급하여 복수로 이용 가능합니다.
기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다.
metadata:
 {'source': '신한은행', 'row': 21, 'type': '홈페이지', 'timestamp': 1701838934.879154, 'id':
'618f6304-66fc-46a8-9322-330fa5707497'}
-----------------------------------------------
2. Chunk: 266 Characters
-----------------------------------------------
ask: 영업점에서 인터넷뱅킹 가입하고 왔는데 홈페이지회원은 별도로 가입해야 하나요?
Information: 인터넷뱅킹 가입과 홈페이지 회원은 개별로 운영되고 있습니다.
인터넷뱅킹만 가입을 하셨다면 별도로 홈페이지에 회원가입 후 홈페이지 로그인이 가능합니다.
※ 영업점에서 인터넷뱅킹 가입 시 전계좌조회서비스 가입하신 고객은 홈페이지 상에서 이용자비밀번호 등록 후 이용가능합니다.
기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다.
metadata:
 {'source': '신한은행', 'row': 23, 'type': '홈페이지', 'timestamp': 1701838934.8791583, 'id':
'000007ec-c333-47bd-94ba-7c2e89e1020e

#### 3. OpenSearch Hybrid 검색

In [32]:
from langchain. retrievers import EnsembleRetriever

- Lexical Search의 경우, search option 변경 가능

In [33]:
filter01 = "홈페이지"
# filter01 = "인증서"
filter02 = "신한은행"
# filter02 = "아마존은행"

opensearch_lexical_retriever.update_search_params(
  k=5,
  minimum_should_match = 0,
  filter = [
    {"term": { "metadata.type": filter01}},
    {"term": {"metadata.source": filter02}}
  ]
)

In [34]:
query = "홈페이지 이용자아이디를 여러 개 사용할 수 있나요?"

In [36]:
ensemble_retriever = EnsembleRetriever(
  retrievers=[opensearch_lexical_retriever, opensearch_semantic_retriever],
  weights=[0.5, 0.5],
  c=100,
  k=5
)

In [41]:
search_hybrid_result = ensemble_retriever.get_relevant_documents(query=query)

answer = chain.run(
  input_documents=search_keyword_result,
  question=query
)

print("#"*80)
print("query: \n", query)
print("answer: \n", answer)

lexical search query: 
{'query': {'bool': {'filter': [],
                    'must': [{'match': {'text': {'minimum_should_match': '0%',
                                                 'operator': 'or',
                                                 'query': '홈페이지 이용자아이디를 여러 개 '
                                                          '사용할 수 있나요?'}}}]}},
 'size': 3}


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:


Human: Here is the context, inside <context></context> XML tags.

<context>
ask: 홈페이지 이용자아이디 여러 개 사용할 수 있나요?
Information: 홈페이지 이용자 아이디는 개인의 경우 1인 1개만 이용 가능하고 기업의 경우에는 1개의 사업자번호당 사용자별로 이용자아이디를 복수로 사용할 수 있습니다. 
※ 개인사업자의 경우 개인과 기업 각각 이용자아이디를 발급하여 복수로 이용 가능합니다. 
기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다.

ask: 영업점에서 인터넷뱅킹 가입하고 왔는데 홈페이지회원은 별도로 가입해야 하나요?
Information: 인터넷뱅킹 가입과 홈페이지 회원은 개별로 운영되고 있습니다. 
인터넷뱅킹만 가입을 하셨다면 별도로 홈페이지에 회원가입 후 홈페이지 로그인이 가능합니다. 
※ 영업점에서 인터넷뱅킹 가입 시 전계좌조회서비스 가입하신 고객은 홈페이지 상에서 이용자비밀번호 등록 후 이용가능합니

In [42]:
show_context_used(search_hybrid_result)

-----------------------------------------------
1. Chunk: 232 Characters
-----------------------------------------------
ask: 홈페이지 이용자아이디 여러 개 사용할 수 있나요?
Information: 홈페이지 이용자 아이디는 개인의 경우 1인 1개만 이용 가능하고 기업의 경우에는 1개의 사업자번호당 사용자별로 이용자아이디를 복수로 사용할 수 있습니다.
※ 개인사업자의 경우 개인과 기업 각각 이용자아이디를 발급하여 복수로 이용 가능합니다.
기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다.
metadata:
 {'source': '신한은행', 'row': 21, 'type': '홈페이지', 'timestamp': 1701838934.879154}
-----------------------------------------------
2. Chunk: 172 Characters
-----------------------------------------------
ask: 홈페이지 회원아이디를 변경하고 싶습니다.
Information: 회원탈퇴를 하시고, 사용하시고 싶은 아이디로 신규 회원가입신청을 하시면 됩니다. 다만, 사용하시고 싶은 아이디가 이미 다른 고객이 사용하고 있는 경우에는
사용하실 수 없습니다. 기타 문의는 콜센터 1599-8000번으로 문의 바랍니다.
metadata:
 {'source': '신한은행', 'row': 45, 'type': '홈페이지', 'timestamp': 1701838934.8792145}
-----------------------------------------------
3. Chunk: 261 Characters
-----------------------------------------------
ask: 아이핀으로 홈페이지회원 가입한 고객은 간편조회서비스 이용할 수 없나요?
Information